In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataset import ECGDataset, read_dataset_from_csv, dataset_folder, read_x_from_csv, read_x_from_mat
from sklearn.metrics import accuracy_score
from torchvision import transforms
from utils import smooth_predictions
from tqdm import tqdm

In [ ]:
TRAIN = True
DATA_CSV = 'data_147.csv'
DATA_PREFIX = DATA_CSV.split('.')[0]

NUM_OF_CLASS = 3


In [ ]:
class ECGModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super(ECGModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(hidden_dim*2, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # Apply classifier to every time step
        time_steps = lstm_out.shape[1]
        out = self.classifier(lstm_out.reshape(-1, lstm_out.shape[2]))
        return out.view(-1, time_steps, self.classifier.out_features)
    



In [ ]:
if TRAIN:
    # Create Dataset and DataLoader 
    # Modify this based on your need 
    dataset = read_dataset_from_csv(train_csv='data/train/s2_src.csv', label_csv='data/train/s2_lbl.csv')
    # dataset = dataset_folder('./data/train')
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    print(f'Dataset len: {len(dataset)}')

    # Initialize the model
    model = ECGModel(input_dim=1, hidden_dim=128, num_layers=2, num_classes=NUM_OF_CLASS)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training the model
    def train(model, dataloader, criterion, optimizer, num_epochs):
        model.train()
        for epoch in range(num_epochs):
            total_loss = 0
            for inputs, labels in dataloader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs.view(-1, 3), labels.view(-1))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f'Epoch {epoch+1}, Average Loss: {total_loss / len(dataloader)}')
            torch.save(model.state_dict(), 'ecg_model.pth')

    # Example training call
    num_epochs = 50
    train(model, dataloader, criterion, optimizer, num_epochs)

In [ ]:
from tqdm import tqdm
model = ECGModel(input_dim=1, hidden_dim=128, num_layers=2, num_classes=NUM_OF_CLASS)
model.load_state_dict(torch.load('ecg_model.pth'))
print("Model loaded")
model.eval() 
# test_dataset = read_dataset_from_csv(train_csv='data/test/s1_src.csv', label_csv='data/test/s1_lbl.csv')
test_dataset = read_x_from_csv(f'data/exp_data/{DATA_CSV}')
# test_dataset = read_x_from_mat('data/exp_data/61b0.mat')
print("Dataset loaded")

test_dataloader = DataLoader(test_dataset, batch_size=16)
def test(model, dataloader):
    all_predictions = []
    all_labels = []
    all_srcs = []
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Testing"):
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 2)  # Get predicted class for each time step
            all_predictions.extend(predicted.numpy().flatten())
            all_labels.extend(labels.numpy().flatten())
            all_srcs.extend(inputs.numpy().flatten())

    return np.array(all_predictions), np.array(all_labels), np.array(all_srcs)

# Run the test
predictions, labels, srcs = test(model, test_dataloader)


smoothed = smooth_predictions(predictions)
accuracy = accuracy_score(labels, predictions)
sm_acc = accuracy_score(labels, smoothed)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Accuracy smoothed: {sm_acc * 100:.2f}%')

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(2, figsize=(12, 3))
start = 3500
end = start + 300
# axs[0].plot(labels[start:end])
# axs[0].set_title('Label')
axs[0].plot(smoothed[start:end]*300)
axs[0].set_title('Prediction')
axs[0].plot(srcs[start:end])
axs[0].set_title('ECG')

In [ ]:
# Plot the signal
start = 300
end = start + 300
fontsize = 24
signal = srcs[start:end]
labels = smoothed[start:end]
plt.figure(figsize=(30, 10))
plt.plot(signal, label='ECG', linewidth=2)
colors = {0: 'white', 1: 'lightblue', 2: 'lightgreen'}
# Shade each ROI region based on the colors defined
segments = np.where(np.diff(labels) != 0)[0]
for i in range(len(segments)-1):
    
    start_x = segments[i]
    end_x = segments[i+1]
    # start_x = smoothed[start]
    # end_x = smoothed[end]
    plt.axvspan(start_x, end_x, color=colors[labels[end_x]], alpha=1)


# Add labels, grid, and a legend
plt.xlabel('Time (s)', fontsize=fontsize)
plt.ylabel('Signal amplitude', fontsize=fontsize)
plt.title('ECG ROIs classification sample result', fontsize=fontsize)
plt.grid(True)
plt.legend()
plt.legend(fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)

plt.show()

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters: {total_params}')

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
def to_roi_df(labels):
    changes = np.diff(labels, prepend=labels[0])

    # Identify the start and end indices
    starts = np.where(changes != 0)[0]
    ends = np.where(changes != 0)[0][1:]

    # Include the end of the last segment
    ends = np.append(ends, len(labels))

    # Get the wave types
    waves = labels[starts]

    # Create the DataFrame
    df = pd.DataFrame({
        'wave': waves,
        'start_index': starts,
        'end_index': ends - 1
    })

    # Filter out the segments with wave type 0
    df = df[df['wave'] != 0].reset_index(drop=True)

    # Map the wave types to their names
    wave_mapping = {2: 'p', 1: 'qrs'}
    df['wave'] = df['wave'].map(wave_mapping)
    df['duration'] = df['end_index'] - df['start_index']
    return df

def find_peaks_in_segment(segment, wave_type):
    plt.show()
    if wave_type == 'p':
        # For 'p' wave, we expect one peak
        peaks, _ = find_peaks(segment)
        if peaks.size == 0:
            return [np.nan]  # No peak found
        return [peaks[np.argmax(segment[peaks])]]  # Return the location of the highest peak
    elif wave_type == 'qrs':
        # For 'qrs' wave, we expect two peaks: R and J
        peaks, _ = find_peaks(segment)
        
        if peaks.size < 2:
            return [np.nan, np.nan]  # Less than 2 peaks found
        sorted_peaks = peaks[np.argsort(segment[peaks])][::-1]  # Sort peaks by height
        return sorted(sorted_peaks[:2])  # Return the two largest peaks in time order

# Function to get peaks from the DataFrame
def get_peaks_from_roi(df, signal):
    peaks_data = {'wave': [], 'peak1': [], 'peak2': [], 'duration':[]}
    
    for index, row in df.iterrows():
        segment = signal[row['start_index']:row['end_index'] + 1]

        peaks = find_peaks_in_segment(segment, row['wave'])
        print(peaks)
        peaks_data['wave'].append(row['wave'])
        peaks_data['peak1'].append(row['start_index'] + peaks[0] if not np.isnan(peaks[0]) else np.nan)
        print(row['start_index'] + peaks[0] if not np.isnan(peaks[0]) else np.nan)
        peaks_data['peak2'].append(row['start_index'] + peaks[1] if len(peaks) > 1 and not np.isnan(peaks[1]) else np.nan)
        peaks_data['duration'].append(row.duration)

    return pd.DataFrame(peaks_data)



def filter_peaks(df):
    valid_indices = []
    i = 0
    while i < len(df) - 1:
        if (df.iloc[i]['wave'] == 'p' and not np.isnan(df.iloc[i]['peak1']) and
            df.iloc[i + 1]['wave'] == 'qrs' and not np.isnan(df.iloc[i + 1]['peak1']) and not np.isnan(df.iloc[i + 1]['peak2'])):
            valid_indices.extend([i, i + 1])
            i += 2
        else:
            i += 1
    return df.iloc[valid_indices].reset_index(drop=True)

roi_df = to_roi_df(smoothed)

In [ ]:
roi_df

In [ ]:
peak_df = get_peaks_from_roi(roi_df, srcs)
peak_df = filter_peaks(peak_df)

In [ ]:
len(signal)

In [ ]:
# peak_df.to_csv(f'{DATA_PREFIX}_peaks.csv')
peak_df

In [ ]:
def calculate_intervals(df, sampling_rate=512):
    intervals = {
        'p_peak_index': [],
        'pr_interval': [],
        'pj_interval': [],
        'duration':[]
    }
    
    i = 0
    while i < len(df) - 1:
        if (df.iloc[i]['wave'] == 'p' and not np.isnan(df.iloc[i]['peak1']) and
            df.iloc[i + 1]['wave'] == 'qrs' and not np.isnan(df.iloc[i + 1]['peak1']) and not np.isnan(df.iloc[i + 1]['peak2'])):
            p_peak_index = df.iloc[i]['peak1']
            qrs_peak1 = df.iloc[i + 1]['peak1']
            qrs_peak2 = df.iloc[i + 1]['peak2']
            
            pr_interval = (qrs_peak1 - p_peak_index) / sampling_rate
            pj_interval = (qrs_peak2 - p_peak_index) / sampling_rate
            print(f'({qrs_peak2} - {p_peak_index})/{sampling_rate} = {pj_interval}')
            print(pj_interval)
            
            intervals['p_peak_index'].append(p_peak_index)
            intervals['pr_interval'].append(pr_interval)
            intervals['pj_interval'].append(pj_interval)
            intervals['duration'].append(df.iloc[i].duration)
            
            i += 2
        else:
            i += 1
    
    return pd.DataFrame(intervals)

intervals = calculate_intervals(peak_df)
intervals['p_peak_index'] = intervals['p_peak_index'].astype(int)

In [ ]:

intervals.set_index('p_peak_index', inplace=True)
ecg_df = pd.DataFrame({"ECG":srcs, "Label":smoothed})
merged_df = ecg_df.merge(intervals, right_on='p_peak_index', left_index=True, how='left')
merged_df = merged_df.ffill().bfill()
merged_df.to_csv(f'data/intervals/{DATA_PREFIX}_intervals.csv')

In [ ]:
p_df = peak_df[peak_df['wave'] == 'p']
p_df['index'] = [int(x) for x in p_df['peak1']]
p_df['ppinterval'] = p_df['peak1'].diff()
p_df

In [ ]:
merged_df = merged_df.merge(p_df[['index', 'ppinterval']], right_on='index', left_index=True, how='left')
merged_df = merged_df.ffill().bfill()

In [ ]:
merged_df.to_csv(f"{DATA_PREFIX}_intervals.csv")